# $\color{violet}{\textbf{Heston Pricer}}$

In [2]:
!pip install QuantLib

     |████████████████████████████████| 20.1 MB 1.3 MB/s 


In [61]:
import QuantLib as ql
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime 
import random
import time
from math import sqrt, exp

In [202]:
def heston_EUCall_price(strike_price, maturity, spot_price, variance, kappa, theta, epsilon, rho):
  # Payoff function inputs are option type and strike price
  option_type = ql.Option.Call
  call_payoff = ql.PlainVanillaPayoff(option_type, strike_price) 

  # Exercise function takes maturity date of the option as input
  day_count = ql.Actual365Fixed()
  calendar = ql.UnitedStates()
  today = ql.Date(1, 1,2021)

  call_exercise = ql.EuropeanExercise(maturity)

  # Function inputs are striked type payoff and exercise
  option = ql.VanillaOption(call_payoff, call_exercise)

  riskfree_rate = 0.0
  dividend = 0

  initial_value = ql.QuoteHandle(ql.SimpleQuote(spot_price))

  # Setting up flat risk free curves
  discount_curve = ql.YieldTermStructureHandle(ql.FlatForward(today, riskfree_rate,day_count))
  dividend_yield = ql.YieldTermStructureHandle(ql.FlatForward(today, dividend, day_count))

  heston_process = ql.HestonProcess(discount_curve,dividend_yield, initial_value,variance,kappa,theta,epsilon,rho)
 
  # Inputs used for the engine are model, Tolerance level, maximum evaluations
  engine = ql.AnalyticHestonEngine(ql.HestonModel(heston_process),0.01,100000000)
  option.setPricingEngine(engine)
  price = option.NPV()
  return price

In [134]:
###################
# test the pricer #
###################

strike_price = 100
T = 364 #no of days to maturity
today = ql.Date(1, 1,2021)
maturity = today+T

# Option input values
spot_price = 105
v0 = 0.01 # Initial variance is square of volatility
kappa = 2       # Speed of mean reversion 
theta = 0.01    # Long-run variance
sigma = 0.1   # Volatility of volatility
rho = 0.0       # Correlation  

t = time.time()
print("option price =", heston_EUCall_price(strike_price, maturity, spot_price, v0, kappa, theta, sigma, rho))
end = time.time()-t
print("elapsed time:", end)

option price = 7.027599518261383
elapsed time: 0.004851341247558594


#  $\color{violet}{\textbf{Making a grid for the parameters}: \kappa\times\theta\times\sigma\times\nu_0\times\rho}$

In [85]:
def make_heston_grid(kappa, theta, sigma, v0, rho):
  grid = np.array([[0,0,0,0,0]])
  n = kappa.shape[0]
  for i in range(n):
    for j in range(n):
      for k in range(n):
        for l in range(n):
          for m in range(n):
            grid = np.concatenate((grid, [[kappa[i], theta[j], sigma[k], v0[l], rho[m]]]), axis=0)
  return pd.DataFrame(np.delete(grid,0,axis=0), columns=['kappa', 'theta', 'sigma', 'v0', 'rho']).round(decimals=2)

In [86]:
######################
# choice of the grid #
######################

kappa = np.linspace(0.01,10,10)
theta = np.linspace(0.01,1,10)
sigma = np.linspace(0.01,1,10)
v0 = np.linspace(0.01,1,10)
rho = np.linspace(-.95,.95,10)

In [87]:
#################
# run time ~40s #
#################

train = make_heston_grid(kappa, theta, sigma, v0, rho)

In [88]:
train.sample(5)

,kappa,theta,sigma,v0,rho
8277,0.01,0.89,0.23,0.78,0.53
31404,3.34,0.12,0.45,0.01,-0.11
1475,0.01,0.12,0.45,0.78,0.11
33812,3.34,0.34,0.89,0.12,-0.53
360,0.01,0.01,0.34,0.67,-0.95


#  $\color{violet}{\textbf{Creating the responses}}$

In [193]:
########################################
# fix grid for maturities and strikes #
########################################

"""
changes in the vectors T, K according to data
"""

T = np.arange(301, 401, step=10) 
K = np.arange(51, 151, step=10)
grid = np.zeros((T.shape[0]*K.shape[0],2))
grid[:,0] = np.repeat(T,K.shape[0])
grid[:,1] = np.tile(K, T.shape[0])

In [203]:
labels = np.zeros((1, grid.shape[0]))
temp = np.zeros((1, grid.shape[0]))

In [156]:
!pip install tqdm

In [ ]:
"""
pay attention to the spot price
"""

from tqdm import tqdm
for i in tqdm(range(train.shape[0]),colour="red", smoothing=0.5, miniters=0):
  for j in range(labels.shape[1]):
    temp[0,j] = heston_EUCall_price(strike_price = grid[j,1], maturity = today+int(grid[j,0]), spot_price = 100, variance= train["v0"][i], kappa = train["kappa"][i],
                                   theta = train["theta"][i], epsilon = train["sigma"][i], rho = train["rho"][i])
  labels = np.concatenate((labels, temp))  

 16%|█▌        | 16174/100000 [06:54<45:46, 30.52it/s]

In [196]:
labels = np.delete(labels,0,axis=0)

In [ ]:
pd.DataFrame(labels).to_csv("labels.csv", header = False, index = True) 